In [ ]:
import pandas as pd

import json as js

import ast

# new imports from base script
from absl import logging

import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
import tensorflow_hub as hub
import numpy as np
import os
import re

from sklearn.decomposition import PCA

def tables(file):
    
    '''Accepts an Azure Video Indexer JSON file path and returns
    a tuple with a dataframe dictionary in index 0 and a 
    combined dataframe in index 1'''
    
    pd.set_option('display.max_columns', None)

    pd.set_option('display.max_rows', None)
    
    with open(file,'r') as f:
        
        data = js.loads(f.read())
    
        f.close()
    
    df_nested_list_videos = pd.json_normalize(data, record_path = ['videos'])
    
    namelist = ["insights.transcript", "insights.ocr", 
                "insights.keywords", "insights.topics", 
                "insights.labels", "insights.namedLocations", 
                "insights.namedPeople", "insights.audioEffects", 
                "insights.sentiments", "insights.emotions"]
    
    dfdictionary = {}
    
    fulltable = pd.DataFrame()
    
    for name in namelist:
        
        stringjson = df_nested_list_videos[name].to_json(orient = 'records')
        
        stringjson = stringjson.replace(":null", ":\"null\"")

        stringjson = stringjson.replace(":false", ":\"false\"")
        
        stringjson = stringjson.replace(":true", ":\"true\"")
        
        listjson = ast.literal_eval(stringjson)

        json = listjson[0]

        maindf = pd.json_normalize(json)
        
        truncatedname = name[9:]
        
        dfdictionary[truncatedname] = maindf
        
        print(f"Dataframe access: (yourdictionaryname)['{truncatedname}']")
        
        if truncatedname == 'sentiments':
            
            newcolumn = maindf['sentimentType'] + '_' + maindf['averageScore'].astype('str') 
            
            maindf.insert(1, 'column', newcolumn)
            
            maindf = pd.DataFrame(maindf, columns = ['id', 'column', 'instances'])
            
        jsoninstances = maindf['instances'].to_json(orient = 'records')

        listinstances = ast.literal_eval(jsoninstances)
        
        typelist = []

        for i in range(0, len(maindf)):

            typelist.append(truncatedname)
            
        multiple = False
            
        for row in listinstances:

            if len(row) > 1: 

                multiple = True
                
                break

        maindf.insert(0, "insightType", typelist)

        maindf.rename(columns = {(maindf.columns[2]): 'insight'}, inplace = True)

        if 'confidence' in maindf.columns:

            maindf = pd.DataFrame(maindf, columns = ['insightType', 'insight', 'confidence'])

        else:

            maindf = pd.DataFrame(maindf, columns = ['insightType', 'insight'])

        instancesdf = pd.DataFrame()
        
        confidence = 0
        
        if 'confidence' in pd.json_normalize(listinstances[0]).columns:
            
            confidence = 1
            
        elif 'confidence' in maindf.columns:
            
            confidence = 2
            
        else:
            
            confidence = 3

        if multiple == True:

            count = 0

            instancesdf = pd.DataFrame()

            for row in listinstances:

                row = pd.json_normalize(row)
                
                if truncatedname in ['namedLocations', 'namedPeople']:
                
                    row = row.drop(columns = 'instanceSource')

                insighttypelist = []

                insightlist = []

                confidencelist = []

                for i in range(0, len(row)):

                    insighttypelist.append(truncatedname)

                    insightlist.append(maindf.iloc[count, 1])
                    
                    if confidence == 2:

                        confidencelist.append(maindf.iloc[count, 2])
                        
                    elif confidence == 3:
                        
                        confidencelist.append('1.0000')
                        
                columntable = pd.DataFrame()

                columntable.insert(0, 'insightType', insighttypelist)

                columntable.insert(1, 'insight', insightlist)
                
                if (confidence == 2 or confidence == 3):
                    
                    columntable.insert(2, 'confidence', confidencelist)

                instancesdf = pd.concat([instancesdf, pd.concat([columntable, row], axis = 1, ignore_index = True)], ignore_index = True)
                
                count += 1

            multiple = False

        else:
            
            instancesdf = pd.DataFrame()

            for row in listinstances:

                row = pd.json_normalize(row)

                instancesdf = pd.concat([instancesdf, row], ignore_index = True)

            instancesdf = pd.concat([maindf, instancesdf], axis = 1, ignore_index = True)
        
        fulltable = pd.concat([fulltable, instancesdf], ignore_index = True)
        
    fulltable = pd.DataFrame(fulltable, columns = [0, 1, 2, 3, 4])
    
    fulltable.rename(columns = {fulltable.columns[0]: "insightType", fulltable.columns[1]: "insight", 
    fulltable.columns[2]: "confidence", fulltable.columns[3]: "start", fulltable.columns[4]: "end"}, inplace = True)
    
    return (dfdictionary, fulltable)

In [ ]:
dictionary = tables()

dfdictionary = dictionary[0]

fulltable = dictionary[1]

In [ ]:
dfdictionary['transcript']

In [ ]:
fulltable

In [ ]:
fulltable = fulltable.replace(regex = ["Desired Regex Strings"], value = "Redacted")

insightlist = fulltable['insight'].to_list()

print(insightlist)

In [ ]:
logging.set_verbosity(logging.ERROR)

pca = PCA(n_components = 3)

with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    
    insight_embeddings = session.run(embed(insightlist))
    
    xyz_coordinates = pca.fit_transform(np.array(insight_embeddings))
    
    print(xyz_coordinates)

In [ ]:
fulltable = pd.DataFrame(fulltable, columns = ['insightType', 'insight', 'confidence', 'start', 'end'])

x, y, z = (np.array(xyz_coordinates).T)

x = x.tolist();

y = y.tolist();

z = z.tolist();

xyztable = pd.DataFrame()

xyztable.insert(0, 'x', x)

xyztable.insert(1, 'y', y)

xyztable.insert(2, 'z', z)

booleantable = xyztable.duplicated()

print(booleantable)

count = .025

for index, value in booleantable.items():
    
    if value == True:

        xyztable.iloc[[index]] = xyztable.iloc[[index]].add([count, count, count], axis = 'columns')
        
        count += .025
    
    else:

        count = .025

print(xyztable)

fulltable = pd.concat([fulltable, xyztable], axis = 1, ignore_index = True)

fulltable.rename(columns = {fulltable.columns[0]: "insightType", fulltable.columns[1]: "insight", 
    fulltable.columns[2]: "confidence", fulltable.columns[3]: "start", fulltable.columns[4]: "end",
    fulltable.columns[5]: "x", fulltable.columns[6]: "y", fulltable.columns[7]: "z"}, inplace = True)

fulltable

In [ ]:
fulltable.to_csv('Desired Path To Your CSV File', index = False)